This notebook code mostly relies on the implementation of UNIT found here:
https://github.com/eriklindernoren/PyTorch-GAN/tree/master/implementations/unit

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
import numpy as np
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)



class ResidualBlock(nn.Module):
    def __init__(self, features):
        super(ResidualBlock, self).__init__()

        conv_block = [
            nn.ReflectionPad2d(1),
            nn.Conv2d(features, features, 3),
            nn.InstanceNorm2d(features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(features, features, 3),
            nn.InstanceNorm2d(features),
        ]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return x + self.conv_block(x)


class Encoder(nn.Module):
    def __init__(self, in_channels=3, dim=64, n_downsample=2, shared_block=None):
        super(Encoder, self).__init__()

        # Initial convolution block
        layers = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_channels, dim, 7),
            nn.InstanceNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
        ]

        # Downsampling
        for _ in range(n_downsample):
            layers += [
                nn.Conv2d(dim, dim * 2, 4, stride=2, padding=1),
                nn.InstanceNorm2d(dim * 2),
                nn.ReLU(inplace=True),
            ]
            dim *= 2

        # Residual blocks
        for _ in range(3):
            layers += [ResidualBlock(dim)]

        self.model_blocks = nn.Sequential(*layers)
        self.shared_block = shared_block

    def reparameterization(self, mu):
        Tensor = torch.cuda.FloatTensor if mu.is_cuda else torch.FloatTensor
        z = Variable(Tensor(np.random.normal(0, 1, mu.shape)))
        return z + mu

    def forward(self, x):
        x = self.model_blocks(x)
        mu = self.shared_block(x)
        z = self.reparameterization(mu)
        return mu, z


class Generator(nn.Module):
    def __init__(self, out_channels=3, dim=64, n_upsample=2, shared_block=None):
        super(Generator, self).__init__()

        self.shared_block = shared_block

        layers = []
        dim = dim * 2 ** n_upsample
        # Residual blocks
        for _ in range(3):
            layers += [ResidualBlock(dim)]

        # Upsampling
        for _ in range(n_upsample):
            layers += [
                nn.ConvTranspose2d(dim, dim // 2, 4, stride=2, padding=1),
                nn.InstanceNorm2d(dim // 2),
                nn.LeakyReLU(0.2, inplace=True),
            ]
            dim = dim // 2

        # Output layer
        layers += [nn.ReflectionPad2d(3), nn.Conv2d(dim, out_channels, 7), nn.Tanh()]

        self.model_blocks = nn.Sequential(*layers)

    def forward(self, x):
        x = self.shared_block(x)
        x = self.model_blocks(x)
        return x



class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        channels, height, width = input_shape
        # Calculate output of image discriminator (PatchGAN)
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.Conv2d(512, 1, 3, padding=1)
        )

    def forward(self, img):
        return self.model(img)

In [ ]:
import glob
import random
import os

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms


class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=True, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned
        #print(os.path.join(root, "monet_jpg") + "/*.*")
        self.files_A = sorted(glob.glob(os.path.join(root, "monet_jpg") + "/*.*"))
        self.files_B = sorted(glob.glob(os.path.join(root, "photo_jpg") + "/*.*"))

        #print("taille fichier max", max(len(self.files_A), len(self.files_B)))
        #print(sorted(glob.glob(os.path.join(root, "monet_jpg") + "/*.*")))
    def __getitem__(self, index):
        item_A = self.transform(Image.open(self.files_A[index % len(self.files_A)]))

        if self.unaligned:
            item_B = self.transform(Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)]))
        else:
            item_B = self.transform(Image.open(self.files_B[index % len(self.files_B)]))

        return {"monet_jpg": item_A, "photo_jpg": item_B}

    def __len__(self):
        
        return min(len(self.files_A), len(self.files_B))

In [ ]:
import argparse
import os
import numpy as np
import math
import itertools
import datetime
import time
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable


import torch.nn as nn
import torch.nn.functional as F
import torch


cuda = True if torch.cuda.is_available() else False

dataset_name ='../input/gan-getting-started'     # default apple2orange help="name of the dataset" 
n_epochs = 15
dim = 64

b1 = 0.5  #"adam: decay of first order momentum of gradient")
b2 = 0.999
epoch = 0
  

savings = "/content/gdrive/My Drive/deep_project"
batch_size=1
lr = 0.005
b1 = 0.5  #"adam: decay of first order momentum of gradient")
b2 = 0.999
decay_epoch = 2000  
n_cpu = 1     #"number of cpu threads to use during batch generation"
img_height = 256
img_width = 256
channels = 3
sample_interval = 2000 #"interval between saving generator samples"
checkpoint_interval = 1000  #interval between saving model checkpoints


LR = [0.0002] #learning rates
n_downsample = 1 #change to 2 originally

def reverse_normalize(image, mean_=0.5, std_=0.5):
    #credit: https://www.kaggle.com/adrianda/cyclegan-pytorch-style-transfer
    if torch.is_tensor(image):
        image = image.detach().numpy()
    un_normalized_img = image * std_ + mean_
    un_normalized_img = un_normalized_img * 255
    return np.uint8(un_normalized_img)


def sample_images(batches_done):
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))
    #print(imgs)
    X1 = Variable(imgs["monet_jpg"].type(Tensor))
    X2 = Variable(imgs["photo_jpg"].type(Tensor))
    _, Z1 = E1(X1)
    _, Z2 = E2(X2)
    fake_X1 = G1(Z2)
    fake_X2 = G2(Z1)
    
    #Generate grids
    X1 = Tensor.cpu(X2) 
    fake_X1 = Tensor.cpu(fake_X1) 
    #print(X1.shape)
    grid_x =  reverse_normalize(make_grid(X1, nrow=4).permute(1, 2, 0).detach().cpu().numpy())
    grid_fake_y =  reverse_normalize(make_grid(fake_X1, nrow=4).permute(1, 2, 0).detach().cpu().numpy() )
    
    #Transformation from X -> Y

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(30, 20))
    
    #ax1.imshow(X1.permute(1, 2, 0))
    ax1.imshow(grid_x)
    
    ax1.axis('off')
    ax1.set_title('X')
    ax2.imshow(grid_fake_y)
    ax2.axis('off')
    ax2.set_title('Fake Y, Monet-esque')
    plt.show()
    
    
    
    
    
def compute_kl(mu):
    mu_2 = torch.pow(mu, 2)
    loss = torch.mean(mu_2)
    return loss




L = [] # Losses list
for rate in LR:
    criterion_GAN = torch.nn.MSELoss()
    criterion_pixel = torch.nn.L1Loss()

    input_shape = (3, 256, 256)

# Dimensionality (channel-wise) of image embedding
    shared_dim = dim * 2 ** n_downsample

# Initialize generator and discriminator
    shared_E = ResidualBlock(features=shared_dim)
    E1 = Encoder(dim=dim, n_downsample=n_downsample, shared_block=shared_E)
    E2 = Encoder(dim=dim, n_downsample=n_downsample, shared_block=shared_E)
    shared_G = ResidualBlock(features=shared_dim)
    G1 = Generator(dim=dim, n_upsample=n_downsample, shared_block=shared_G)
    G2 = Generator(dim=dim, n_upsample=n_downsample, shared_block=shared_G)
    D1 = Discriminator(input_shape)
    D2 = Discriminator(input_shape)

    if cuda:
        E1 = E1.cuda()
        E2 = E2.cuda()
        G1 = G1.cuda()
        G2 = G2.cuda()
        D1 = D1.cuda()
        D2 = D2.cuda()
        criterion_GAN.cuda()
        criterion_pixel.cuda()


    E1.apply(weights_init_normal)
    E2.apply(weights_init_normal)
    G1.apply(weights_init_normal)
    G2.apply(weights_init_normal)
    D1.apply(weights_init_normal)
    D2.apply(weights_init_normal)

# Loss weights
    lambda_0 = 10  # GAN
    lambda_1 = 0.1  # KL (encoded images)
    lambda_2 = 100  # ID pixel-wise
    lambda_3 = 0.1  # KL (encoded translated images)
    lambda_4 = 100  # Cycle pixel-wise

# Optimizers
    optimizer_G = torch.optim.Adam(
        itertools.chain(E1.parameters(), E2.parameters(), G1.parameters(), G2.parameters()),
        lr=rate,
        betas=(b1, b2),
    )
    optimizer_D1 = torch.optim.Adam(D1.parameters(), lr=rate, betas=(b1, b2))
    optimizer_D2 = torch.optim.Adam(D2.parameters(), lr=rate, betas=(b1, b2))

# Learning rate update schedulers
    lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
        optimizer_G, lr_lambda=LambdaLR(n_epochs, 0, 0).step
    )
    lr_scheduler_D1 = torch.optim.lr_scheduler.LambdaLR(
        optimizer_D1, lr_lambda=LambdaLR(n_epochs, 0, 0).step
    )
    lr_scheduler_D2 = torch.optim.lr_scheduler.LambdaLR(
        optimizer_D2, lr_lambda=LambdaLR(n_epochs, 0, 0).step
    )

    Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

# Image transformations
    transforms_ = [
        transforms.Resize(int(256 * 1.12), Image.BICUBIC),
        transforms.RandomCrop((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]



# ----------
#  Training
# ----------
    losses = []
    prev_time = time.time()
    for epoch in range(0, n_epochs):
        torch.manual_seed(epoch)
        
        average_loss_G1 = 0
        av_loss = 0
        
        # Training data loader
        dataloader = DataLoader(
            ImageDataset("%s" % dataset_name, transforms_=transforms_, unaligned=True),
            batch_size=batch_size,
            shuffle=True,
            num_workers=n_cpu,       #changer à 1 ????
        )
        # Test data loader
        val_dataloader = DataLoader(
            ImageDataset("%s" % dataset_name, transforms_=transforms_, unaligned=True, mode="test"),
            batch_size=5,
            shuffle=True,
            num_workers=1,
        )
        for i, batch in enumerate(dataloader):

        # Set model input
            X1 = Variable(batch["monet_jpg"].type(Tensor))
            X2 = Variable(batch["photo_jpg"].type(Tensor))

        # Adversarial ground truths
            valid = Variable(Tensor(np.ones((X1.size(0), *D1.output_shape))), requires_grad=False)
            fake = Variable(Tensor(np.zeros((X1.size(0), *D1.output_shape))), requires_grad=False)

        # -------------------------------
        #  Train Encoders and Generators
        # -------------------------------

            optimizer_G.zero_grad()

        # Get shared latent representation
            mu1, Z1 = E1(X1)
            mu2, Z2 = E2(X2)

        # Reconstruct images
            recon_X1 = G1(Z1)
            recon_X2 = G2(Z2)

        # Translate images
            fake_X1 = G1(Z2)
            fake_X2 = G2(Z1)

        # Cycle translation
            mu1_, Z1_ = E1(fake_X1)
            mu2_, Z2_ = E2(fake_X2)
            cycle_X1 = G1(Z2_)
            cycle_X2 = G2(Z1_)

        # Losses
            loss_GAN_1 = lambda_0 * criterion_GAN(D1(fake_X1), valid)
            loss_GAN_2 = lambda_0 * criterion_GAN(D2(fake_X2), valid)
            loss_KL_1 = lambda_1 * compute_kl(mu1)
            loss_KL_2 = lambda_1 * compute_kl(mu2)
            loss_ID_1 = lambda_2 * criterion_pixel(recon_X1, X1)
            loss_ID_2 = lambda_2 * criterion_pixel(recon_X2, X2)
            loss_KL_1_ = lambda_3 * compute_kl(mu1_)
            loss_KL_2_ = lambda_3 * compute_kl(mu2_)
            loss_cyc_1 = lambda_4 * criterion_pixel(cycle_X1, X1)
            loss_cyc_2 = lambda_4 * criterion_pixel(cycle_X2, X2)

        # Total loss
            loss_G = (
                loss_KL_1
                + loss_KL_2
                + loss_ID_1
                + loss_ID_2
                + loss_GAN_1
                + loss_GAN_2
                + loss_KL_1_
                + loss_KL_2_
                + loss_cyc_1
                + loss_cyc_2
            )

            loss_G.backward()
            optimizer_G.step()
        
            av_loss += loss_GAN_1 /len(dataloader)
            average_loss_G1 += loss_G /(len(dataloader))
        
        # -----------------------
        #  Train Discriminator 1
        # -----------------------

            optimizer_D1.zero_grad()

            loss_D1 = criterion_GAN(D1(X1), valid) + criterion_GAN(D1(fake_X1.detach()), fake)

            loss_D1.backward()
            optimizer_D1.step()

        # -----------------------
        #  Train Discriminator 2
        # -----------------------

            optimizer_D2.zero_grad()

            loss_D2 = criterion_GAN(D2(X2), valid) + criterion_GAN(D2(fake_X2.detach()), fake)

            loss_D2.backward()
            optimizer_D2.step()

        # --------------
        #  Log Progress
        # --------------
####
        # Determine approximate time left
            batches_done = epoch * len(dataloader) + i
            batches_left = n_epochs * len(dataloader) - batches_done
            time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
            prev_time = time.time()

        # Print log
            sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] ETA: %s"
                % (epoch, n_epochs, i, len(dataloader), (loss_D1 + loss_D2).item(), loss_G.item(), time_left)
            )
            if batches_done % sample_interval == 0:
                sample_images(batches_done)


        losses.append(average_loss_G1.item())
        print("\n averageloss: ",average_loss_G1.item())
        print("gan1_loss", av_loss.item())
        # Update learning rates
        lr_scheduler_G.step()
        lr_scheduler_D1.step()
        lr_scheduler_D2.step()
    L.append(losses)

In [ ]:
import PIL
! rm -rf ../images
! mkdir ../images

In [ ]:
import PIL
from PIL import Image
import imageio
from tqdm import tqdm


class ImageDataset(Dataset):
        """
        Custom dataset
        """
        
        def __init__(self, img_path, img_size=256, normalize=True):
            self.img_path = img_path
            
            if normalize:
                self.transform = transforms.Compose([
                    transforms.Resize(img_size),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5], std=[0.5])
                ])
            else:
                self.transform = transforms.Compose([
                    transforms.Resize(img_size),
                    transforms.ToTensor()
                ])
            
            #Dictionary entries
            self.img_idx = dict()
            for number_, img_ in enumerate(os.listdir(self.img_path)):
                self.img_idx[number_] = img_
                
        def __len__(self):
            #Length of dataset --> number of images
            return len(self.img_idx)
        
        def __getitem__(self, idx):
            img_path = os.path.join(self.img_path, self.img_idx[idx])
            img = Image.open(img_path)
            img = self.transform(img)
            
            return img

        
E1.eval()        
G1.eval()

path_photo = "../input/gan-getting-started/photo_jpg"
#Get data loader for final transformation / submission
dataset_photo = ImageDataset(path_photo, img_size=256, normalize=True)
submit_dataloader = DataLoader(dataset_photo, batch_size=1, shuffle=False)


dataiter = iter(submit_dataloader)

#Previous normalization choosen
mean_=0.5 
std_=0.5

#Loop through each picture
for image_idx in tqdm(range(0, len(submit_dataloader))):
        
    #Get base picture
    fixed_X = dataiter.next()
    
    #Identify correct device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    #Create fake pictures (monet-esque)
    #X1 = Variable(imgs["monet_jpg"].type(Tensor))
    _,encod_fake = E2(fixed_X.to(device))
    fake_Y = G1(encod_fake)
    fake_Y = fake_Y.detach().cpu().numpy()
    fake_Y = reverse_normalize(fake_Y, mean_, std_)
    fake_Y = fake_Y[0].transpose(1, 2, 0)
    fake_Y = np.uint8(fake_Y)
    fake_Y = Image.fromarray(fake_Y)
    #print(fake_Y.shape)
    
    #Save picture
    fake_Y.save("../images/" + str(image_idx) + ".jpg")

#Back to it
        
E1.train()        
G1.train()

In [ ]:
import shutil
shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")